In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.express as px

newnames_reco = {"aice_body-sharp":"DLR", 
                 "first_body":"MBIR", 
                 "fbp_fc08":"FBP", 
                 "aidr3d_fc08":"HIR", 
                 "asir_standard":"HIR", 
                 "fbp_standard":"FBP",
                 "qr40f-q3_0.4":"Qr40f-Q3_0.4",
                 "qr40f-q3_0.6":"Qr40f-Q3_0.6",
                 "qr40f-q3_1":"Qr40f-Q3_1",
                 "br40f-q3_0.4":"Br40f-Q3_0.4",
                 "br40f-q3_0.6":"Br40f-Q3_0.6",
                 "br40f-q3_1":"Br40f-Q3_1"}

newnames_ct = {
    "CT4": "<b>Vendor 1",
    "GE": "<b>Vendor 2",
    "photon":"<b>Vendor 3",
}

patient_phantom_networks = ["3d_fullres_LiTS+phantom_246", "3d_fullres_LiTS+phantom_247", "3d_fullres_LiTS+phantom_249", "3d_fullres_LiTS+phantom_250",
                            "3d_fullres_LiTS+phantom_254", "3d_fullres_LiTS+phantom_255", "3d_fullres_LiTS+phantom_256", "3d_fullres_LiTS+phantom_257",
                            "3d_fullres_LiTS+phantom_258", "3d_fullres_LiTS+phantom_259"]

dice_detection_threshold = 0
size_filter = 4/3 * 3.14159265359 * (5/2)**3   # in cubic millimeters, 5mm sphere, ignore all predictions smaller than this

output_dir = '../../plots/3) deep learning performance eval/network_improvement'


In [ ]:
df_lesion_meta = pd.read_csv('results_csv/results_lesion_paper.csv')
df_lesion_meta = df_lesion_meta[df_lesion_meta["network"].isin(patient_phantom_networks)]

df_lesion_meta.loc[(df_lesion_meta["volume_pred"] < size_filter) & (df_lesion_meta["volume_gt"].notna()),"volume_pred"] = None
df_lesion_meta.loc[(df_lesion_meta["volume_pred"] < size_filter) & (df_lesion_meta["volume_gt"].notna()),"dice"] = None
df_lesion_meta = df_lesion_meta[(df_lesion_meta["volume_pred"] > size_filter) | (df_lesion_meta["volume_pred"].isna()) ]

df_lesion_meta["detected"] = df_lesion_meta["dice"] > dice_detection_threshold
filter_lesion_props = (df_lesion_meta["C in HU"] < -10) | (df_lesion_meta["C in HU"].isna())
df_lesion_meta = df_lesion_meta[filter_lesion_props]
df_lesion_meta = df_lesion_meta[df_lesion_meta["kernel"] != "FC13"]

df_lesion_meta["ct"] = df_lesion_meta["ct"].map(newnames_ct)
df_lesion_meta = df_lesion_meta[df_lesion_meta["date"].isin([230915, 231205, 240306])]

In [ ]:
df_lesion_meta["lesion_prop"] = df_lesion_meta.apply(lambda x: f"{str(2*int(x['R in mm'])).zfill(2)}mm/{int(x['C in HU'])}HU" if not np.isnan(x['R in mm']) else None , axis=1)

df_lesion_meta["reco+kernel"] = df_lesion_meta["reco"].str.lower() + "_" + df_lesion_meta["kernel"].str.lower()
df_lesion_meta["reco+kernel"] = df_lesion_meta["reco+kernel"].map(newnames_reco)


In [ ]:
groupby_lesion_prop = ["network", "CTDIvol", "reco+kernel", "ct", "lesion_prop", "R in mm", "C in HU"]
groupby_lesion_prop_rep = ["network", "CTDIvol", "reco+kernel", "ct", "lesion_prop", "R in mm", "C in HU", "rep"]
groupby_lesion_all = ["network", "CTDIvol", "reco+kernel", "ct"]

In [ ]:
df_lesion_meta_recall = df_lesion_meta[~df_lesion_meta["filename_gt"].isna()] 

df_lesion_meta_recall = df_lesion_meta_recall.groupby(groupby_lesion_all).agg({"detected": ["mean", "sum", "count"]}).reset_index()
df_lesion_meta_recall.columns = groupby_lesion_all + ["recall", "num_detected", "num_total"]

df_lesion_meta["FP"] = df_lesion_meta["filename_gt"].isna() & df_lesion_meta["filename_pred"].notna() & (df_lesion_meta["px_numb_pred"] >= 0)

df_lesion_meta_precision = df_lesion_meta.copy()

df_lesion_meta_precision = df_lesion_meta_precision.groupby(groupby_lesion_all).agg({"detected": ["mean", "sum", "count"], "dice":["mean", "std"], "FP":"sum"}).reset_index()
df_lesion_meta_precision.columns = groupby_lesion_all + ["precision", "num_detected", "num_total", "dice_mean", "dice_std", "FP"]

lesion_seg_summary = df_lesion_meta_recall.merge(df_lesion_meta_precision, on=groupby_lesion_all, how="left")
lesion_seg_summary["F1-score"] = 2 * (lesion_seg_summary["precision"] * lesion_seg_summary["recall"]) / (lesion_seg_summary["precision"] + lesion_seg_summary["recall"])
lesion_seg_summary["F1-score"] = lesion_seg_summary["F1-score"].fillna(0)

In [ ]:
df_lesion_meta_recall_rep = df_lesion_meta[~df_lesion_meta["filename_gt"].isna()] 
df_lesion_meta_recall_rep = df_lesion_meta_recall_rep.groupby(groupby_lesion_all + ["rep"]).agg({"detected": ["mean", "sum", "count"]}).reset_index()
df_lesion_meta_recall_rep.columns = groupby_lesion_all + ["rep"] + ["recall", "num_detected", "num_total"]

df_lesion_meta_precision_rep = df_lesion_meta.copy()
df_lesion_meta_precision_rep = df_lesion_meta_precision_rep.groupby(groupby_lesion_all + ["rep"]).agg({"detected": ["mean", "sum", "count"], "dice":["mean", "std"], "FP":"sum"}).reset_index()
df_lesion_meta_precision_rep.columns = groupby_lesion_all + ["rep"] + ["precision", "num_detected", "num_total", "dice_mean", "dice_std", "FP"]

lesion_seg_summary_rep = df_lesion_meta_recall_rep.merge(df_lesion_meta_precision_rep, on=groupby_lesion_all + ["rep"], how="left")
lesion_seg_summary_rep["F1-score"] = 2 * (lesion_seg_summary_rep["precision"] * lesion_seg_summary_rep["recall"]) / (lesion_seg_summary_rep["precision"] + lesion_seg_summary_rep["recall"])
lesion_seg_summary_rep["F1-score"] = lesion_seg_summary_rep["F1-score"].fillna(0)



In [ ]:
lesion_seg_summary_rep_mean = lesion_seg_summary_rep.groupby(groupby_lesion_all).agg({"F1-score": "mean", "recall": "mean", "precision": "mean", "num_detected_x": "mean", "num_total_x": "mean", "dice_mean": "mean", "dice_std": "mean"}).reset_index()
lesion_seg_summary_rep_mean.columns = groupby_lesion_all + ["F1-score", "recall", "precision", "num_detected", "num_total", "dice_mean", "dice_std"]

In [ ]:
df_lesion_meta_recall_rep_lesion_prop = df_lesion_meta[~df_lesion_meta["filename_gt"].isna()] 
df_lesion_meta_recall_rep_lesion_prop = df_lesion_meta_recall_rep_lesion_prop.groupby(groupby_lesion_prop_rep).agg({"detected": ["mean", "sum", "count"]}).reset_index()
df_lesion_meta_recall_rep_lesion_prop.columns = groupby_lesion_prop_rep + ["recall", "num_detected", "num_total"]

df_lesion_meta_recall_rep_lesion_prop_mean = df_lesion_meta_recall_rep_lesion_prop.groupby(groupby_lesion_prop).agg({"recall": ["min", "mean", "std"], "num_detected": ["mean", "std"], "num_total": ["mean", "std"]}).reset_index()
df_lesion_meta_recall_rep_lesion_prop_mean.columns = groupby_lesion_prop + ["recall_min", "recall_mean", "recall_std", "num_detected_mean", "num_detected_std", "num_total_mean", "num_total_std"]


In [ ]:
lesion_seg_summary["CTDIvol"] = lesion_seg_summary["CTDIvol"].astype(float).round(1).astype(str)
lesion_seg_summary_rep["CTDIvol"] = lesion_seg_summary_rep["CTDIvol"].astype(float).round(1).astype(str)

In [ ]:
lesion_seg_summary_rep_plot = lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"] == "<b>Vendor 1")]
lesion_seg_summary_rep_plot =  lesion_seg_summary_rep_plot[lesion_seg_summary_rep_plot["reco+kernel"].isin(["HIR"])]

fig = px.strip(lesion_seg_summary_rep_plot,
             x="CTDIvol", y="F1-score",
             color="network", facet_row="ct", 
             title="F1-score for lesion detection", range_y=[0,1], template="simple_white", 
)    

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score", col=1)

fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,
    width=1100
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_canon_improved_points_dicetreshold_{dice_detection_threshold}.png"))
